In [1]:
# coding: utf-8
import pandas as pd
import json
import traceback

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
keywords = {}
most_related_wordscore = {}

In [3]:
with open('./physics_direct_words.json') as f:
    direct_words = json.load(f, encoding='utf-8')
    direct_words_dict = dict((el,True) for el in direct_words)
with open('./physics_related_words.json') as f:
    related_words = json.load(f, encoding='utf-8')
with open('./other_subjects_nonphysics.json') as f:
    other_words = json.load(f, encoding='utf-8')
    other_words_dict = dict((el,True) for el in other_words)

In [4]:
content = None
for file in range(20):
    print("file", file)
    with open('./data/mag_papers_%d.txt' % file) as f:
        content = f.readlines()

        data = []
        for line in content:
            obj = json.loads(line)
            if 'fos' in obj:
                data.append(obj)

        for obj in data:
            for i in range(len(obj['fos'])):
                obj['fos'][i] = obj['fos'][i].lower()
                if obj['fos'][i] not in keywords:
                    keywords[obj['fos'][i]] = 0
                if obj['fos'][i] not in most_related_wordscore:
                    most_related_wordscore[obj['fos'][i]] = 0

        for i, obj in enumerate(data):
            if i%100000==0:
                print(i)

            fos = obj['fos']
            direct = 0
            other = 0
            related = 0

            for lword in fos:

                if lword in direct_words_dict:
                    direct += 1
                elif lword in other_words_dict:
                    other += 1

                if most_related_wordscore[lword] == 0:
                    e = process.extract(lword, related_words, limit=1)
                    score = e[0][1]
                    most_related_wordscore[lword] = score
                else:
                    score = most_related_wordscore[lword]

                if score >= 85:
                    related += 1

            if direct > other or (direct == other and related*2 >= len(fos)):
                # This is the related subject
                for lword in fos:
                    keywords[lword] += 1
            elif direct < other:
                if related*1.5 >= len(fos):
                    # Although it is not this subject but there are much of related keywords
                    for lword in fos:
                        keywords[lword] -= 0.5
                else:
                    # It is not this subject also there are lack of related keywords
                    for lword in fos:
                        keywords[lword] -= 1
            else:
                # It may be this subject but there are lack of related keywords
                for lword in fos:
                    keywords[lword] += 0.5

file 0
0
100000
200000
300000
400000
500000
600000
file 1
0
100000
200000
300000
400000
500000
600000
file 2
0
100000
200000
300000
400000
500000
600000
file 3
0
100000
200000
300000
400000
500000
600000
file 4
0
100000
200000
300000
400000
500000
600000
file 5
0
100000
200000
300000
400000
500000
600000
file 6
0
100000
200000
300000
400000
500000
600000
file 7
0
100000
200000
300000
400000
500000
600000
file 8
0
100000
200000
300000
400000
500000
600000
file 9
0
100000
200000
300000
400000
500000
600000
file 10
0
100000
200000
300000
400000
500000
600000
file 11
0
100000
200000
300000
400000
500000
600000
file 12
0
100000
200000
300000
400000
500000
600000
file 13
0
100000
200000
300000
400000
500000
600000
file 14
0
100000
200000
300000
400000
500000
600000
file 15
0
100000
200000
300000
400000
500000
600000
file 16
0
100000
200000
300000
400000
500000
600000
file 17
0
100000
200000
300000
400000
500000
600000
file 18
0
100000
200000
300000
400000
500000
600000
file 19
0
100000
20000

In [5]:
import operator
sorted_x = sorted(keywords.items(), key=operator.itemgetter(1))
sorted_x.reverse()
sorted_x

[('physics', 844304.0),
 ('quantum mechanics', 349924.0),
 ('optics', 127212.5),
 ('condensed matter physics', 106340.0),
 ('classical mechanics', 96126.5),
 ('atomic physics', 91404.5),
 ('nuclear magnetic resonance', 89970.5),
 ('thermodynamics', 67309.5),
 ('astrophysics', 67175.0),
 ('nuclear physics', 66824.0),
 ('quantum electrodynamics', 64652.5),
 ('astronomy', 57250.5),
 ('particle physics', 54987.5),
 ('political science', 49643.0),
 ('cardiology', 31165.5),
 ('optoelectronics', 24028.0),
 ('statistical physics', 23545.5),
 ('magnetic field', 23358.0),
 ('environmental science', 21201.0),
 ('thin film', 18255.5),
 ('mechanics', 13280.0),
 ('computational chemistry', 12615.0),
 ('spectrum', 12547.0),
 ('oscillation', 12188.5),
 ('medical education', 11774.5),
 ('cross section', 11433.0),
 ('acoustics', 10573.0),
 ('electric field', 10206.5),
 ('superconductivity', 10066.0),
 ('theoretical physics', 9270.0),
 ('heat transfer', 9000.0),
 ('astrobiology', 8815.0),
 ('ground state

In [6]:
with open('./data/physics_keywords.json', 'w') as f:
    json.dump(keywords, f, ensure_ascii=False)

In [28]:
df = pd.DataFrame.from_dict(keywords, orient='index').reset_index()
df.rename(columns=dict(zip(df.columns[[0, 1]], ['keyword', 'score'])),inplace=True)
df = df.set_index('keyword')

In [32]:
df = df.sort_values(by=['score'], ascending=False)

In [33]:
df.loc[df['score'] > 0]

,score
keyword,
physics,844304.0
quantum mechanics,349924.0
optics,127212.5
condensed matter physics,106340.0
classical mechanics,96126.5
atomic physics,91404.5
nuclear magnetic resonance,89970.5
thermodynamics,67309.5
astrophysics,67175.0


In [194]:
keywords['electrostatics']

-1289.0